## 5.2.4

In [14]:
function initialize_spins(Lx,Ly)
    return rand([-1,1],Lx,Ly)
end

initialize_spins (generic function with 1 method)

In [2]:
function measure_Mz(Ck)
    return sum(Ck)
end

measure_Mz (generic function with 1 method)

In [3]:
function measure_energy(Ck,J,h,Lx,Ly)
    energy = 0
    for iy=1:Ly
        for ix=1:Lx
            Si = calc_Si(ix,iy,Lx,Ly,Ck)
            σi = Ck[ix,iy]
            energy += -(J/2)*σi*Si - h*σi
        end
    end
    return energy
end

measure_energy (generic function with 1 method)

In [4]:
function calc_Si(ix,iy,Lx,Ly,Ck)
    jx = ix + 1
    if jx > Lx
        jx -= Lx
    end
    jy=iy
    Si = Ck[jx,jy]
    
    jx = ix - 1
    if jx < 1
        jx += Lx
    end
    jy = iy
    Si += Ck[jx,jy]
    
    jy = iy + 1
    if jy > Ly
        jy -= Ly
    end
    jx = ix
    Si += Ck[jx,jy]
    
    jy = iy-1
    if jy < 1
        jy += Ly
    end
    jx = ix
    Si += Ck[jx,jy]
    return Si
end

calc_Si (generic function with 1 method)

In [5]:
function calc_ΔE(Ck,ix,iy,J,h,Lx,Ly)
    Si = calc_Si(ix,iy,Lx,Ly,Ck)
    return 2J*Ck[ix,iy]*Si + 2h*Ck[ix,iy]
end

calc_ΔE (generic function with 1 method)

In [6]:
function metropolis(σi,ΔE,T)
    is_accepted = ifelse(rand() <= exp(-ΔE/T),true,false)
    σ_new = ifelse(is_accepted,-σi,σi)
    return σ_new,is_accepted
end

metropolis (generic function with 1 method)

In [7]:
function heatbath(σi,ΔE,T)
    α = ΔE*σi
    σ_new = ifelse(rand() <= 1/(1+exp(-α/T)),+1,-1)
    is_accepted = ifelse(σ_new == σi,false,true)
    return σ_new,is_accepted
end


heatbath (generic function with 1 method)

In [8]:
function local_metropolis_update(Ck,ix,iy,T,J,h,Lx,Ly)
    ΔE = calc_ΔE(Ck,ix,iy,J,h,Lx,Ly)
    σi = Ck[ix,iy]
    return metropolis(σi,ΔE,T)
end
function local_heatbath_update(Ck,ix,iy,T,J,h,Lx,Ly)
    ΔE = calc_ΔE(Ck,ix,iy,J,h,Lx,Ly)
    σi = Ck[ix,iy]
    return heatbath(σi,ΔE,T)
end

local_heatbath_update (generic function with 1 method)

In [9]:
using Random
using Plots
function montecarlo(num_thermal,num_MC,measure_interval,T,J,h,Lx,Ly)
    Random.seed!(123)
    num_total = num_thermal+num_MC
    accept_count = 0
    absmz_meanvalue = 0
    measure_count = 0
    mz_data = []
    update(Ck,ix,iy) = local_metropolis_update(Ck,ix,iy,T,J,h,Lx,Ly)
    
    Ck = initialize_spins(Lx,Ly)
    
    for trj = 1:num_total
        for isweep = 1:Lx*Ly
            ix = rand(1:Lx)
            iy = rand(1:Ly)
            Ck[ix,iy],is_accepted = update(Ck,ix,iy)
            
            accept_count += ifelse(is_accepted,1,0)
        end
        
        if trj > num_thermal
            if trj % measure_interval == 0
                measure_count += 1
                mz = measure_Mz(Ck)/(Lx*Ly)
                absmz_meanvalue += abs(mz)
                push!(mz_data,mz)
            end
        end 
    end
    return mz_data,accept_count/(num_total*Lx*Ly),absmz_meanvalue/measure_count
                
end

montecarlo (generic function with 1 method)

In [10]:
function test()
    Lx = 100
    Ly = 100
    J = 1
    h = 0
    num_thermal = 200
    num_MC = 10000-num_thermal
    measure_interval = 10
    T = 1
    @time mz_data,acceptance_ratio,absmz = montecarlo(num_thermal,num_MC,measure_interval,T,J,h,Lx,Ly)
    println("average acceptance ratio ",acceptance_ratio)
    histogram(mz_data,bin=-1:0.01:1)
    savefig("mz_data_$T.png")
    return
end
test()

  5.040704 seconds (1.01 k allocations: 145.461 KiB)
average acceptance ratio 0.00181302


In [11]:
function montecarlo_fast(num_thermal,num_MC,measure_interval,T,J,h,Lx,Ly)
    Random.seed!(123)
    num_total = num_thermal+num_MC
    accept_count = 0
    absmz_meanvalue = 0
    measure_count = 0
    mz_data = []
    update(Ck,ix,iy) = local_metropolis_update(Ck,ix,iy,T,J,h,Lx,Ly)
    
    Ck = initialize_spins(Lx,Ly)
    
    for trj = 1:num_total
        for ix = 1:Lx
            for iy=1:Ly
                Ck[ix,iy],is_accepted = update(Ck,ix,iy)
            
                accept_count += ifelse(is_accepted,1,0)
            end
        end
        
        if trj > num_thermal
            if trj % measure_interval == 0
                measure_count += 1
                mz = measure_Mz(Ck)/(Lx*Ly)
                absmz_meanvalue += abs(mz)
                push!(mz_data,mz)
            end
        end 
    end
    return mz_data,accept_count/(num_total*Lx*Ly),absmz_meanvalue/measure_count
                
end

montecarlo_fast (generic function with 1 method)

In [12]:
function test_tdep()
    Lx = 100
    Ly = 100
    J = 1
    h = 0
    num_thermal = 5000
    num_MC = 50000-num_thermal
    measure_interval = 10
    mz_Tdep = []
    nT = 20
    Ts = range(0.5,4.0,length = nT)
    for T in Ts
        @time mz_data,acceptance_ratio,absmz = montecarlo_fast(num_thermal,num_MC,measure_interval,T,J,h,Lx,Ly)
        push!(mz_Tdep,absmz)
        println("$T $absmz")
        histogram(mz_data,bin=-1:0.01:1)
        savefig("mz_data_$(T).png")
    end
    plot(Ts,mz_Tdep)
    savefig("mz_tdep.png")
    return
end

test_tdep (generic function with 1 method)

In [13]:
test_tdep()

 18.427934 seconds (4.52 k allocations: 277.336 KiB)
0.5 0.05199453333333343
 18.698643 seconds (4.52 k allocations: 277.336 KiB)
0.6842105263157895 0.23100075555555571
 18.489597 seconds (4.52 k allocations: 277.336 KiB)
0.868421052631579 0.9997946222222384
 18.601177 seconds (4.52 k allocations: 277.336 KiB)
1.0526315789473684 0.9988989777777715
 18.372596 seconds (4.52 k allocations: 277.336 KiB)
1.236842105263158 0.9962966666666621
 18.643263 seconds (4.52 k allocations: 277.336 KiB)
1.4210526315789473 0.9755179555555478
 18.486626 seconds (4.52 k allocations: 277.336 KiB)
1.605263157894737 0.9791694222222268
1031.705697 seconds (4.52 k allocations: 277.336 KiB)
1.7894736842105263 0.9584731555555526
 18.315030 seconds (4.52 k allocations: 277.336 KiB)
1.9736842105263157 0.9195799555555548
 18.242392 seconds (4.52 k allocations: 277.336 KiB)
2.1578947368421053 0.8301916000000004
 18.593858 seconds (4.52 k allocations: 277.336 KiB)
2.3421052631578947 0.16207084444444436
 18.476981 se